In [1]:
import json
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from credentials import *

In [2]:
"""Establecer acceso a la API de Twitter."""
CONSUMER_KEY = "uvcMjAH4FJglTG8QS9zlpJC01"
CONSUMER_SECRET = 'Ed2c2FhXzpHFagbfTVb3gTnziNTQ3hDk0UDRiiraPfupO5At0s'
ACCESS_TOKEN = "1442475234913775619-hKJkC4Pp3N0ztkvGAue6niVGtra8YZ"
ACCESS_SECRET = "QqW0LjrlYr1YrkllT1RhMbi4LmUvARaecltG6HTzAobUf"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)
print("authenticated")

authenticated


In [3]:
tampico_madero = [-97.87777, 22.28519, -96.87777, 23.28519, -97.83623, 22.27228, -96.83623, 23.27228]

CONVERTIR TWEETS

In [4]:
import pandas as pd
import numpy as np

In [7]:
# In[]:
"""Definir la clase que permitirá el uso del Twitter Streaming API."""
#Twitter Streaming API nos permite descargar mensajes de twitter en tiempo real
class listener(tweepy.StreamListener):
    
    def __init__(self, numero_tweets):
        self.received_tweets_counter = 0
        self.max_number_tweets = numero_tweets
        #Directorio y nombre del archivo de texto donde guardaremos los tweets
        self.file = open('C:/Users/jdroj/Downloads/BigData/TweetsMap/tweets_tampico_madero.csv', 'a', encoding="UTF-8")
        super(listener, self).__init__()
    
    def on_data(self, data):
        if (self.received_tweets_counter < self.max_number_tweets):
            
            self.received_tweets_counter += 1
            #La API de Twitter devuelve datos en formato JSON,
            #asi que hay que decodificarlos.
            try:
                decoded = json.loads(data)
            except Exception as e:
                print(e)
                return True
            #No todos los usuarios tienen habilitada la opcion de geolocalizacion
            #Por ello hay que dar formato a cuando no este disponible
            if decoded.get('geo') is not None:                
                location = str(decoded.get('geo').get('coordinates'))
            else:
                location = '[,]'
            
            #Extraer los datos que nos interese de los tweets
            text = decoded['text'].replace('\n',' ')
            user = '@' + decoded.get('user').get('screen_name')
            created = decoded.get('created_at')
            
            #Escribir los tweets en el archivo de texto
            self.file.write(user + "|" + location + "|" + created + "|" + text + "\n")
            
            return True
        else:
            self.file.close()
            print('Done!')
            return False

    #def on_status(self, status):
        #print(status.text)
        
    def on_error(self, status):
        print(status)
        #Debido a que el uso de la API de twitter tiene un limite diario debemos
        #desconectarnos del stream cuando excedamos dicho limite
        if status == 420:
            print('status code 420: ' + status)
            self.file.close()
            #Retornando un False en on_error Conseguimos desconectarnos del Stream
            return False
        self.file.close()

In [8]:
if __name__ == '__main__':
    print('Starting...')
    #Crear un Stream, estableciendo el número de tweets que se desean guardar
    twitterStream = tweepy.Stream(auth, listener(numero_tweets = 50))
    #Iniciar un Stream (Puede tomar mucho tiempo en terminar)
    #Mas info. en: https://developer.twitter.com/en/docs/tweets/filter-realtime/guides/basic-stream-parameters
    twitterStream.filter(locations = tampico_madero)

Starting...
Done!


In [9]:
tweets_raw = pd.read_table('C:/Users/jdroj/Downloads/BigData/TweetsMap/tweets_tampico_madero.csv', header = None, iterator=False)
tweets_2 = pd.DataFrame()
tweets_raw

,0
0,"@stefaniiepmc|[,]|Tue Nov 16 05:06:47 +0000 20..."


In [ ]:
#stamp = Timestamp('1/1/2014 16:20', tz='America/Sao_Paulo')
#new_stamp = stamp.tz_convert('US/Eastern')

In [40]:
while 1:
    #Debido a que el archivo de texto de los tweets es demasiado grande
    #debe de ser procesado por chunks en lugar de cargar todo el archivo en memoria
    tweets = tweets_raw.get_chunk(500) #1000 filas por chunk
    tweets.columns = ['tweets']
    tweets['len'] = tweets.tweets.apply(lambda x: len(x.split('|')))
    tweets[tweets.len < 4] = np.nan
    del tweets['len']
    tweets = tweets[tweets.tweets.notnull()]    
    
    #Establecer las columnas y valores que tomarán
    tweets['user'] = tweets.tweets.apply(lambda x: x.split('|')[0])
    tweets['geo'] = tweets.tweets.apply(lambda x: x.split('|')[1])
    tweets['timestamp'] = tweets.tweets.apply(lambda x: x.split('|')[2])
    tweets['tweet'] = tweets.tweets.apply(lambda x: x.split('|')[3])
    tweets['lat'] = tweets.geo.apply(lambda x: x.split(',')[0].replace('[',''))
    tweets['lon'] = tweets.geo.apply(lambda x: x.split(',')[1].replace(']',''))
    del tweets['tweets']
    del tweets['geo']
    
    #Convertir las latitudes y longitudes de string a float
    tweets['lon'] = pd.to_numeric(tweets['lon'], downcast="float")
    tweets['lat'] = pd.to_numeric(tweets['lat'], downcast="float")
    
    #Cambiar la zona horaria de UTC a GMT-5
    tweets['timestamp'] = pd.to_datetime(tweets['timestamp'], utc = False)
    stamp = tweets['timestamp']
    #tweets = tweets.set_index('timestamp').dt.tz_convert('America/Mexico_City').reset_index()
    tweets = tweets['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/Mexico_City')
    #Almacenar los tweets en el dataframe
    tweets_2 = tweets_2.append(tweets, ignore_index = True)
    #Guardar los tweets en un archivo CSV
    tweets.to_csv('C:/Users/jdroj/Downloads/BigData/TweetsMap/tweets_tampico_madero.csv', mode='a', header=False, index=False)


AttributeError: 'DataFrame' object has no attribute 'get_chunk'